# SFHo/SFHx example for O$_2$sclpy

See the O$_2$sclpy documentation at https://neutronstars.utk.edu/code/o2sclpy for more information.

In [2]:
import o2sclpy
import matplotlib.pyplot as plot
import ctypes
import numpy
import sys

plots=True
if 'pytest' in sys.modules:
    plots=False

Link the O$_2$scl library:

In [3]:
link=o2sclpy.linker()
link.link_o2scl()

Get the value of $\hbar c$ from an O$_2$scl find_constants object:

In [4]:
fc=o2sclpy.find_constants(link)
hc=fc.find_unique('hbarc','MeV*fm')
print('hbarc = %7.6e' % (hc))

hbarc = 1.973270e+02


Get a copy (a pointer to) the O$_2$scl unit conversion object:

In [5]:
cu=link.o2scl_settings.get_convert_units()

In [6]:
sfho=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfho,'SFHo')
sfho_old=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfho_old,'SFHo_old')
sfhx=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfhx,'SFHx')
sfhx_old=o2sclpy.eos_had_rmf(link)
o2sclpy.rmf_load(link,sfhx_old,'SFHx_old')

Compute nuclear saturation and output the saturation density
and binding energy:

In [7]:
sfho.saturation()
print('SFHo: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV' % 
      (sfho.n0,sfho.eoa*hc,sfho.comp*hc,sfho.msom,sfho.esym*hc,
       sfho.fesym_slope(sfho.n0)*hc))
print('')

SFHo: n0=1.582415e-01 1/fm^3, E/A=-1.617240e+01 MeV, K=2.452197e+02 MeV, M*/M=7.610394e-01, S=3.123132e+01 MeV, L=4.574132e+01 MeV



In [8]:
sfho_old.saturation()
print('SFHo_old: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV' % 
      (sfho_old.n0,sfho_old.eoa*hc,sfho_old.comp*hc,sfho_old.msom,sfho_old.esym*hc,
       sfho_old.fesym_slope(sfho_old.n0)*hc))
print('')

SFHo_old: n0=1.582673e-01 1/fm^3, E/A=-1.610892e+01 MeV, K=2.430965e+02 MeV, M*/M=7.614850e-01, S=3.179160e+01 MeV, L=4.522591e+01 MeV



In [9]:
sfhx.saturation()
print('SFHx: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV' % 
      (sfhx.n0,sfhx.eoa*hc,sfhx.comp*hc,sfhx.msom,sfhx.esym*hc,
       sfhx.fesym_slope(sfhx.n0)*hc))
print('')

SFHx: n0=1.600292e-01 1/fm^3, E/A=-1.614755e+01 MeV, K=2.386343e+02 MeV, M*/M=7.181083e-01, S=2.827242e+01 MeV, L=2.318892e+01 MeV



In [10]:
sfhx_old.saturation()
print('SFHx_old: n0=%7.6e 1/fm^3, E/A=%7.6e MeV, K=%7.6e MeV, M*/M=%7.6e, S=%7.6e MeV, L=%7.6e MeV' % 
      (sfhx_old.n0,sfhx_old.eoa*hc,sfhx_old.comp*hc,sfhx_old.msom,sfhx_old.esym*hc,
       sfhx_old.fesym_slope(sfhx_old.n0)*hc))
print('')

SFHx_old: n0=1.625773e-01 1/fm^3, E/A=-1.643793e+01 MeV, K=2.438174e+02 MeV, M*/M=7.034481e-01, S=2.830425e+01 MeV, L=2.431376e+01 MeV

